<a href="https://colab.research.google.com/github/WelzJR/Cryptography-BID-Auction/blob/main/Cryptography_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# === Global setup ===
import os
import secrets
import hashlib

# Optional: if you want everything saved in Drive, uncomment:
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/mpc_auction_project')

# Prime field for secret sharing & commitments
P = 2**127 - 1  # big prime

print("Using prime field P =", P)


Using prime field P = 170141183460469231731687303715884105727


In [22]:
!pip install tinyec


In [23]:
%%writefile commitments.py
import secrets
from tinyec import registry

def generate_commitment_params():
    """
    Generate public parameters for an elliptic-curve Pedersen commitment.
    Using curve secp256r1.
    """
    curve = registry.get_curve("secp256r1")
    G = curve.g

    # Choose random non-zero scalar k and define H = k * G
    k = secrets.randbelow(curve.field.n)
    while k == 0:
        k = secrets.randbelow(curve.field.n)
    H = k * G

    return {
        "curve": curve,
        "G": G,
        "H": H,
    }


def commit_bid(bid: int, randomness: int, params):
    """
    EC Pedersen commitment:
        C = bid * G + randomness * H
    """
    curve = params["curve"]
    G = params["G"]
    H = params["H"]
    n = curve.field.n

    bid = bid % n
    randomness = randomness % n

    C = bid * G + randomness * H
    return C


def verify_commitment(commitment, bid: int, randomness: int, params) -> bool:
    C_check = commit_bid(bid, randomness, params)
    return (C_check.x == commitment.x) and (C_check.y == commitment.y)


Overwriting commitments.py


In [24]:
%%writefile zkproofs.py
import hashlib
from commitments import commit_bid, verify_commitment

def _hash_to_int(*args, modulus: int):
    """
    Hash arbitrary values to an integer modulo 'modulus'.
    """
    h = hashlib.sha256()
    for a in args:
        h.update(str(a).encode("utf-8"))
    return int.from_bytes(h.digest(), "big") % modulus


def generate_range_proof(bid: int, randomness: int, B_min: int, B_max: int, params):
    """
    Simplified *pedagogical* non-interactive range 'proof'.

    IMPORTANT NOTE (for the report / TA):
    -------------------------------------
    - This construction is only a toy and not a production-grade
      zero-knowledge range proof.
    - It does NOT put the raw 'bid' value into the proof object.
    - The verifier will learn the bid via an out-of-band opening
      (bid, randomness) in our simulation, but the proof structure
      itself does not explicitly contain it.

    Idea:
      c = H(C, B_min, B_max)          (Fiat–Shamir-style challenge)
      z = bid + c * randomness (mod n)

    proof = { 'c': c, 'z': z }

    In a real ZK protocol, the verifier would be able to check that
    bid is in range without learning bid, by using more elaborate
    range proofs. Here we keep things simple for the assignment.
    """
    curve = params["curve"]
    n = curve.field.n

    if not (B_min <= bid <= B_max):
        raise ValueError(f"Bid {bid} out of range [{B_min}, {B_max}]")

    # Compute the commitment that this proof relates to
    C = commit_bid(bid, randomness, params)

    # Fiat–Shamir challenge
    c = _hash_to_int(C.x, C.y, B_min, B_max, modulus=n)

    # Response
    z = (bid + c * randomness) % n

    proof = {
        "c": c,
        "z": z,
    }
    return proof


def verify_range_proof(commitment, bid: int, randomness: int,
                       proof: dict, B_min: int, B_max: int, params) -> bool:
    """
    Verify the toy range 'proof'.

    Steps:
      1. Check that commitment corresponds to (bid, randomness).
      2. Recompute c = H(C, B_min, B_max).
      3. Check that proof['c'] equals recomputed c.
      4. Check that z = bid + c * randomness (mod n).
      5. Check range B_min <= bid <= B_max.

    NOTE:
      - This still leaks 'bid' to the verifier through the arguments
        (bid, randomness), so it is NOT a true zero-knowledge proof.
      - However, the proof object itself does not contain 'bid', and
        the structure mimics a non-interactive protocol using
        Fiat–Shamir.
    """
    curve = params["curve"]
    n = curve.field.n

    # 1) Commitment consistency
    if not verify_commitment(commitment, bid, randomness, params):
        return False

    # 2) Recompute challenge
    c_expected = _hash_to_int(commitment.x, commitment.y, B_min, B_max, modulus=n)
    c = proof.get("c")
    z = proof.get("z")

    if c is None or z is None:
        return False
    if c != c_expected:
        return False

    # 3) Check z relation
    if z % n != (bid + c * randomness) % n:
        return False

    # 4) Check range
    if not (B_min <= bid <= B_max):
        return False

    return True


Overwriting zkproofs.py


In [25]:
%%writefile secretsharing.py
import secrets

def share_value(x: int, p: int):
    """
    Generate 3 additive shares x1, x2, x3 of x modulo p:
        x = x1 + x2 + x3 (mod p)
    """
    x1 = secrets.randbelow(p)
    x2 = secrets.randbelow(p)
    x3 = (x - x1 - x2) % p
    return x1, x2, x3


def reconstruct(shares, p: int) -> int:
    """
    Reconstruct x from a list/tuple of additive shares modulo p.
    """
    return sum(shares) % p


Overwriting secretsharing.py


In [26]:
%%writefile mpc.py
from secretsharing import reconstruct

def mpc_add(a_shares, b_shares, p: int):
    """
    Additively add two secret-shared values a, b:
        c = a + b (mod p)
    Given:
        a_shares = [a1, a2, a3]
        b_shares = [b1, b2, b3]
    Returns:
        c_shares = [c1, c2, c3]
    """
    return [ (ai + bi) % p for ai, bi in zip(a_shares, b_shares) ]


def mpc_sub(a_shares, b_shares, p: int):
    """
    Subtract two secret-shared values:
        c = a - b (mod p)
    """
    return [ (ai - bi) % p for ai, bi in zip(a_shares, b_shares) ]


def mpc_mult(a_shares, b_shares, p: int):
    """
    Very simplified 'multiplication' of shared values:
    Here we just multiply shares locally:
        c_i = a_i * b_i (mod p)

    NOTE: This is NOT a correct secure MPC multiplication protocol.
    It's included just to have the function present as in the spec.
    We don't actually need it in this project pipeline.
    """
    return [ (ai * bi) % p for ai, bi in zip(a_shares, b_shares) ]


def _to_signed(x: int, p: int) -> int:
    """
    Interpret x in [0, p-1] as a signed integer in (-(p//2), ..., p//2].
    """
    if x > p // 2:
        return x - p
    return x


def mpc_compare(a_shares, b_shares, p: int):
    """
    Simplified MPC comparison using share differences:
    1. Each server computes delta_i = a_i - b_i (mod p)
    2. They reconstruct delta = sum(delta_i) mod p      (in real MPC this is interactive)
    3. Interpret delta as signed: if delta > 0 => a > b, else a < b

    Returns:
        1  if a > b
       -1  if a < b
        0  if a == b
    """
    # Compute difference shares
    diff_shares = mpc_sub(a_shares, b_shares, p)

    # Reconstruct full delta
    delta_mod = reconstruct(diff_shares, p)
    delta = _to_signed(delta_mod, p)

    if delta > 0:
        return 1
    elif delta < 0:
        return -1
    else:
        return 0


def mpc_argmax(bidder_ids, shares_per_bidder, p: int):
    """
    Find the maximum bid among all bidders using mpc_compare.

    Args:
        bidder_ids: list of bidder identifiers
        shares_per_bidder: dict {bidder_id: [s1, s2, s3]}
        p: modulus

    Returns:
        (winner_id, winning_bid_value)
    """
    if not bidder_ids:
        raise ValueError("No bidders")

    # Start with the first bidder as the current max
    current_winner = bidder_ids[0]

    for b_id in bidder_ids[1:]:
        cmp_res = mpc_compare(shares_per_bidder[current_winner],
                              shares_per_bidder[b_id],
                              p)
        if cmp_res == -1:  # current_winner < b_id
            current_winner = b_id

    # Reconstruct winning bid (allowed to be revealed by the protocol)
    from secretsharing import reconstruct
    winning_bid = reconstruct(shares_per_bidder[current_winner], p)

    return current_winner, winning_bid


Overwriting mpc.py


In [27]:
%%writefile auction.py
from commitments import generate_commitment_params, commit_bid
from zkproofs import generate_range_proof, verify_range_proof
from secretsharing import share_value, reconstruct
from mpc import mpc_compare, mpc_argmax

class AuctionServer:
    """
    Represents one of the three MPC servers (S1, S2, S3).
    Stores additive shares of bids.
    """
    def __init__(self, server_id: int, p: int):
        self.server_id = server_id
        self.p = p
        # bidder_id -> share_value (for this server only)
        self.shares = {}

    def receive_shares(self, bidder_id, share_value: int):
        """
        Server-side function:
        receive_shares(bidder_id, share_value)
        Stores the share securely.
        """
        self.shares[bidder_id] = share_value


# --- Server-side helper functions required by spec ---

def compute_difference(a_share: int, b_share: int, p: int) -> int:
    """
    Local difference on shares:
        diff_share = a_share - b_share (mod p)
    """
    return (a_share - b_share) % p


def reconstruct_difference(diff_shares, p: int) -> int:
    """
    Reconstruct the full difference from diff_shares using additive reconstruction.
    """
    return reconstruct(diff_shares, p)


def compare(a_shares, b_shares, p: int) -> int:
    """
    compare(a_shares, b_shares, p) -> -1 or 1 or 0
    Thin wrapper around mpc_compare.
    """
    return mpc_compare(a_shares, b_shares, p)


def find_max_bid(bidder_ids, shares_per_bidder, p: int):
    """
    find_max_bid(bidders, shares, p) -> (winner_id, winning_bid)
    Thin wrapper around mpc_argmax.
    """
    return mpc_argmax(bidder_ids, shares_per_bidder, p)


# --- Auction logic: bidder registration & running the auction ---
def register_bidder(bidder_id: int,
                    bid: int,
                    p: int,
                    B_min: int,
                    B_max: int,
                    params,
                    servers: list):
    """
    Bidder-side logic (simulated here):
    1) Generate randomness
    2) Create EC Pedersen commitment
    3) Generate toy non-interactive range proof
    4) Verify range proof (servers' side)
    5) If valid -> secret-share bid and send shares to servers
    """
    import secrets
    curve = params["curve"]
    n = curve.field.n

    # 1) Generate randomness mod curve order
    randomness = secrets.randbelow(n)
    while randomness == 0:
        randomness = secrets.randbelow(n)

    # 2) Commitment (EC Pedersen)
    commitment = commit_bid(bid, randomness, params)

    # 3) Generate toy range proof
    proof = generate_range_proof(bid, randomness, B_min, B_max, params)

    # 4) Servers verify the proof
    if not verify_range_proof(commitment, bid, randomness, proof, B_min, B_max, params):
        print(f"[Bidder {bidder_id}] Range proof verification FAILED. Bid rejected.")
        return False

    # 5) Secret-sharing of the bid (over prime field p)
    from secretsharing import share_value
    shares = share_value(bid, p)

    # Distribute shares to servers
    for server, share in zip(servers, shares):
        server.receive_shares(bidder_id, share)

    print(f"[Bidder {bidder_id}] Bid {bid} accepted and shares distributed.")
    return True



def run_auction(servers: list, p: int):
    """
    Run the auction using the shares stored on each server.
    Steps:
    1) Gather the list of bidders (assumed identical on all servers)
    2) Build a dict bidder_id -> [s1, s2, s3]
    3) Use mpc_argmax / find_max_bid to obtain (winner_id, winning_bid)
    """
    if len(servers) != 3:
        raise ValueError("This auction requires exactly 3 servers.")

    # 1) Assume all servers have the same bidder IDs
    bidder_ids = list(servers[0].shares.keys())
    if not bidder_ids:
        raise ValueError("No bids received.")

    # 2) Collect shares per bidder
    shares_per_bidder = {}
    for b_id in bidder_ids:
        shares_per_bidder[b_id] = [
            servers[0].shares[b_id],
            servers[1].shares[b_id],
            servers[2].shares[b_id],
        ]

    # 3) Find max via MPC
    winner_id, winning_bid = find_max_bid(bidder_ids, shares_per_bidder, p)

    return winner_id, winning_bid


Overwriting auction.py


In [28]:
from commitments import generate_commitment_params
from auction import AuctionServer, register_bidder, run_auction

# Reuse the prime P from the setup cell
p = P
params = generate_commitment_params()


# Bidding range
B_MIN = 100
B_MAX = 1000

def simulate_auction(bids):
    print("\n=== New Auction ===")
    print("Bids:", bids)

    # Create 3 servers
    servers = [AuctionServer(i, p) for i in range(1, 4)]

    # Register each bidder
    for bidder_id, bid in enumerate(bids, start=1):
        try:
            register_bidder(bidder_id, bid, p, B_MIN, B_MAX, params, servers)
        except ValueError as e:
            print(f"[Bidder {bidder_id}] Error: {e}")

    # Filter out bidders that were not registered (no shares)
    # auction.run_auction already assumes those with shares
    winner_id, winning_bid = run_auction(servers, p)
    print(f"Result: Winner = {winner_id}, Winning Bid = {winning_bid}")
    return winner_id, winning_bid

# Example 1: Bidders = 150, 920, 600
simulate_auction([150, 920, 600])

# Example 2: Bidders = 350, 350, 100, 300
simulate_auction([350, 350, 100, 300])

# Example 3: Bidders = 10, 999, 300, 700, 2000
simulate_auction([10, 999, 300, 700, 2000])



=== New Auction ===
Bids: [150, 920, 600]
[Bidder 1] Bid 150 accepted and shares distributed.
[Bidder 2] Bid 920 accepted and shares distributed.
[Bidder 3] Bid 600 accepted and shares distributed.
Result: Winner = 2, Winning Bid = 920

=== New Auction ===
Bids: [350, 350, 100, 300]
[Bidder 1] Bid 350 accepted and shares distributed.
[Bidder 2] Bid 350 accepted and shares distributed.
[Bidder 3] Bid 100 accepted and shares distributed.
[Bidder 4] Bid 300 accepted and shares distributed.
Result: Winner = 1, Winning Bid = 350

=== New Auction ===
Bids: [10, 999, 300, 700, 2000]
[Bidder 1] Error: Bid 10 out of range [100, 1000]
[Bidder 2] Bid 999 accepted and shares distributed.
[Bidder 3] Bid 300 accepted and shares distributed.
[Bidder 4] Bid 700 accepted and shares distributed.
[Bidder 5] Error: Bid 2000 out of range [100, 1000]
Result: Winner = 2, Winning Bid = 999


(2, 999)

TEST CASES

In [29]:
simulate_auction([100, 200, 300])
# Basic


=== New Auction ===
Bids: [100, 200, 300]
[Bidder 1] Bid 100 accepted and shares distributed.
[Bidder 2] Bid 200 accepted and shares distributed.
[Bidder 3] Bid 300 accepted and shares distributed.
Result: Winner = 3, Winning Bid = 300


(3, 300)

In [30]:
simulate_auction([50, 150, 900, 1200])
# Mix valid we invalid


=== New Auction ===
Bids: [50, 150, 900, 1200]
[Bidder 1] Error: Bid 50 out of range [100, 1000]
[Bidder 2] Bid 150 accepted and shares distributed.
[Bidder 3] Bid 900 accepted and shares distributed.
[Bidder 4] Error: Bid 1200 out of range [100, 1000]
Result: Winner = 3, Winning Bid = 900


(3, 900)

In [31]:
simulate_auction([500, 500, 500])
# kolo equal


=== New Auction ===
Bids: [500, 500, 500]
[Bidder 1] Bid 500 accepted and shares distributed.
[Bidder 2] Bid 500 accepted and shares distributed.
[Bidder 3] Bid 500 accepted and shares distributed.
Result: Winner = 1, Winning Bid = 500


(1, 500)

In [32]:
simulate_auction([20, 40, 700])
# wa7da bas valid


=== New Auction ===
Bids: [20, 40, 700]
[Bidder 1] Error: Bid 20 out of range [100, 1000]
[Bidder 2] Error: Bid 40 out of range [100, 1000]
[Bidder 3] Bid 700 accepted and shares distributed.
Result: Winner = 3, Winning Bid = 700


(3, 700)

In [33]:
simulate_auction([150, 250, 350, 450, 550, 650, 750, 850])
# Large list


=== New Auction ===
Bids: [150, 250, 350, 450, 550, 650, 750, 850]
[Bidder 1] Bid 150 accepted and shares distributed.
[Bidder 2] Bid 250 accepted and shares distributed.
[Bidder 3] Bid 350 accepted and shares distributed.
[Bidder 4] Bid 450 accepted and shares distributed.
[Bidder 5] Bid 550 accepted and shares distributed.
[Bidder 6] Bid 650 accepted and shares distributed.
[Bidder 7] Bid 750 accepted and shares distributed.
[Bidder 8] Bid 850 accepted and shares distributed.
Result: Winner = 8, Winning Bid = 850


(8, 850)

In [34]:
# worst Case : invalids
simulate_auction([5, 20, 999, 5001, 777])



=== New Auction ===
Bids: [5, 20, 999, 5001, 777]
[Bidder 1] Error: Bid 5 out of range [100, 1000]
[Bidder 2] Error: Bid 20 out of range [100, 1000]
[Bidder 3] Bid 999 accepted and shares distributed.
[Bidder 4] Error: Bid 5001 out of range [100, 1000]
[Bidder 5] Bid 777 accepted and shares distributed.
Result: Winner = 3, Winning Bid = 999


(3, 999)

In [35]:
simulate_auction([-10, 0, 300])
# -ve we 0


=== New Auction ===
Bids: [-10, 0, 300]
[Bidder 1] Error: Bid -10 out of range [100, 1000]
[Bidder 2] Error: Bid 0 out of range [100, 1000]
[Bidder 3] Bid 300 accepted and shares distributed.
Result: Winner = 3, Winning Bid = 300


(3, 300)

In [36]:
# Edge Boundry
simulate_auction([100, 1000, 999, 101])



=== New Auction ===
Bids: [100, 1000, 999, 101]
[Bidder 1] Bid 100 accepted and shares distributed.
[Bidder 2] Bid 1000 accepted and shares distributed.
[Bidder 3] Bid 999 accepted and shares distributed.
[Bidder 4] Bid 101 accepted and shares distributed.
Result: Winner = 2, Winning Bid = 1000


(2, 1000)

In [37]:
# Mix Order
simulate_auction([450, 900, 300, 1000, 999, 250])



=== New Auction ===
Bids: [450, 900, 300, 1000, 999, 250]
[Bidder 1] Bid 450 accepted and shares distributed.
[Bidder 2] Bid 900 accepted and shares distributed.
[Bidder 3] Bid 300 accepted and shares distributed.
[Bidder 4] Bid 1000 accepted and shares distributed.
[Bidder 5] Bid 999 accepted and shares distributed.
[Bidder 6] Bid 250 accepted and shares distributed.
Result: Winner = 4, Winning Bid = 1000


(4, 1000)